# BPE分词

流程：
- 初始化：把所有的单词拆分为单词字符表
- 统计连续两个字符的个数
- 选择频率最大的字符对，加入词汇表，同时合并两个字符对到原单词字符表
- 循环2，3，直到到达最大的循环次数，或者原单词字符表的每一个单词都已经合并成一个整体

实现：
- 使用SentencePiece库
- 调huggingface模型

简单实现如下

In [107]:
import os
import chardet
import numpy as np
from collections import defaultdict, Counter

In [108]:
def get_vocab(corpus):
    rst_vocab = Counter()
    vocab = Counter()
    for word in corpus:
        word = " ".join(list(word)) + "</w>"
        vocab[word] += 1
        for char in word.split():
            rst_vocab[char] += 1
    return rst_vocab, vocab
get_vocab(['thanks', 'thank', 'error', 'china', 'chinese', 'thanksgiving', 'thanksgiving', 'thankyou'])

(Counter({'n': 9,
          'h': 7,
          'i': 6,
          't': 5,
          'a': 5,
          'k': 4,
          's': 3,
          'e': 2,
          'r': 2,
          'o': 2,
          'c': 2,
          'g': 2,
          'v': 2,
          'g</w>': 2,
          's</w>': 1,
          'k</w>': 1,
          'r</w>': 1,
          'a</w>': 1,
          'e</w>': 1,
          'y': 1,
          'u</w>': 1}),
 Counter({'t h a n k s g i v i n g</w>': 2,
          't h a n k s</w>': 1,
          't h a n k</w>': 1,
          'e r r o r</w>': 1,
          'c h i n a</w>': 1,
          'c h i n e s e</w>': 1,
          't h a n k y o u</w>': 1}))

In [109]:
# 寻找字符对
def get_pairs(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

get_pairs(get_vocab(['thanks', 'thank', 'error', 'china', 'chinese', 'thanksgiving', 'thanksgiving', 'thankyou'])[1])

defaultdict(int,
            {('t', 'h'): 5,
             ('h', 'a'): 5,
             ('a', 'n'): 5,
             ('n', 'k'): 4,
             ('k', 's</w>'): 1,
             ('n', 'k</w>'): 1,
             ('e', 'r'): 1,
             ('r', 'r'): 1,
             ('r', 'o'): 1,
             ('o', 'r</w>'): 1,
             ('c', 'h'): 2,
             ('h', 'i'): 2,
             ('i', 'n'): 4,
             ('n', 'a</w>'): 1,
             ('n', 'e'): 1,
             ('e', 's'): 1,
             ('s', 'e</w>'): 1,
             ('k', 's'): 2,
             ('s', 'g'): 2,
             ('g', 'i'): 2,
             ('i', 'v'): 2,
             ('v', 'i'): 2,
             ('n', 'g</w>'): 2,
             ('k', 'y'): 1,
             ('y', 'o'): 1,
             ('o', 'u</w>'): 1})

In [110]:
def merge_vocab(pairs, rst_vocab, vocab):
    # best是一个元组
    best = max(pairs, key=pairs.get)
    best_freq = pairs[best]
    
    # 原来的词组里的单词频率减少
    rst_vocab[best[0]] -= best_freq
    rst_vocab[best[1]] -= best_freq
    
    # 变成比如 "t h"
    word_blank = " ".join(best)
    # "th"
    word_no_blank = "".join(best)
    # 替换原来的vocab
    new_vocab = defaultdict(int)
    rst_vocab[word_no_blank] += best_freq
    for word in vocab:
        new_word = word.replace(word_blank, word_no_blank)
        new_vocab[new_word] = vocab[word]
    
    return rst_vocab, new_vocab

In [111]:
def bpe(corpus, num_merge):
    rst_vocab, vocab = get_vocab(corpus)
    print(f"initial rst_vocab: {rst_vocab}")
    for i in range(num_merge):
        pairs = get_pairs(vocab)
        if len(pairs) < 2:
            break
        rst_vocab, vocab = merge_vocab(pairs, rst_vocab, vocab)
        print(f"epoch: {i+1}, rst_vocab: {rst_vocab}\n pairs: {pairs}\n")
    return rst_vocab, vocab

In [112]:
corpus  = ['older', 'older', 'old', 'fine', 'fine', 'finest', 'finest', 'finest', 'finest', 'finest', 'lower', 'lowest', 'lower', 'mind',
           'cat', 'cats', 'dogs', 'dogs', 'dogs', 'dog']
num_merges = 100

In [113]:
rst_vocab, vocab = bpe(corpus, num_merges)
print(rst_vocab)

initial rst_vocab: Counter({'o': 10, 'e': 10, 'i': 8, 'n': 8, 'f': 7, 't</w>': 7, 'l': 6, 'd': 6, 's': 6, 'r</w>': 4, 's</w>': 4, 'w': 3, 'g': 3, 'd</w>': 2, 'e</w>': 2, 'c': 2, 'a': 2, 'm': 1, 't': 1, 'g</w>': 1})
epoch: 1, rst_vocab: Counter({'o': 10, 'e': 10, 'in': 8, 'f': 7, 't</w>': 7, 'l': 6, 'd': 6, 's': 6, 'r</w>': 4, 's</w>': 4, 'w': 3, 'g': 3, 'd</w>': 2, 'e</w>': 2, 'c': 2, 'a': 2, 'm': 1, 't': 1, 'g</w>': 1, 'i': 0, 'n': 0})
 pairs: defaultdict(<class 'int'>, {('o', 'l'): 3, ('l', 'd'): 2, ('d', 'e'): 2, ('e', 'r</w>'): 4, ('l', 'd</w>'): 1, ('f', 'i'): 7, ('i', 'n'): 8, ('n', 'e</w>'): 2, ('n', 'e'): 5, ('e', 's'): 6, ('s', 't</w>'): 6, ('l', 'o'): 3, ('o', 'w'): 3, ('w', 'e'): 3, ('m', 'i'): 1, ('n', 'd</w>'): 1, ('c', 'a'): 2, ('a', 't</w>'): 1, ('a', 't'): 1, ('t', 's</w>'): 1, ('d', 'o'): 4, ('o', 'g'): 3, ('g', 's</w>'): 3, ('o', 'g</w>'): 1})

epoch: 2, rst_vocab: Counter({'o': 10, 'e': 10, 't</w>': 7, 'fin': 7, 'l': 6, 'd': 6, 's': 6, 'r</w>': 4, 's</w>': 4, 'w': 3,

# WordPiece

BPE在合并的时候，是选取出现频率最高的对进行和并，而WordPiece不是

> 但是好像频率高的，似然概率也高，从公式可以看出

$$
log P(结合后的词) = \sum_{i=1}^{n} logP(字词i)
$$

所以WordPiece其实就是在选取字符对的时候不一样

流程：
- 划分corpus为char级别的vocab
- 利用LM计算句子的似然概率值
- 合并后重新计算似然概率值
- 重复2，3，直到增长概率


# Unigram

基于概率模型的分词器

- 一开始会给定一个单词的不同子分词，每一个子分词对应一个概率
- 接下来将子字词组合成单词，并计算其概率
- 选择不同组合中概率最高的组合，并使用EM算法调整其他字词的概率
- 重复上述过程，同时概率很低的字词会被移除词表